In [1]:

import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
import re
import jieba

C:\Users\dongy\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# 去掉空行应该在数据清理之后

In [2]:
# 数据整合，输出新数据
def merge_file(file1,file2):
    feature = pd.read_csv(file1,sep=",")
    label = pd.read_csv(file2,sep=",")
    data = feature.merge(label,on='id') # 按照id列整合两个训练集表
    data["X"] = data[["title","content"]].apply(lambda x:"".join([str(x[0]),str(x[1])]),axis=1) # "title","content"两列合为一列"X"
    df = data[['id', 'title', 'content', 'label', 'X']]
    return df


data = merge_file("Train_DataSet.csv","Train_DataSet_Label.csv")

In [3]:
# 停用词加载和清洗
def stopwords_list(filepath):
    stopwords=[line.strip() for line in open(filepath,'r',encoding='utf-8').readlines()]
    return stopwords

stopwords=stopwords_list('stopwords.txt')

def stop_clean(strs):
    strs = [w for w in strs if not w in stopwords]
    return strs

# 只保留汉字
def character_save(strs):
    strs = re.findall(r'[\u4e00-\u9fa5]',strs)
    return strs

In [118]:
#处理训练集，将训练集的文本信息和label信息合并，清洗特殊符合，同时将文本内容进行分词
def data_pre_handle(df):
    
    dataDropNa=data
    
#    dfDropNa["X"]=dataDropNa["X"].apply(lambda x: str(x).replace("\\n","").replace(".","").replace("\n","").replace("　","").replace("↓","").replace("/","").replace("|","").replace(" ",""))
    dataDropNa["X_split"]=dataDropNa["X"]
    dataDropNa["X_split"]=dataDropNa["X_split"].apply(lambda x:"".join(character_save(x)))
    dataDropNa["X_split"]=dataDropNa["X_split"].apply(lambda x:" ".join(jieba.cut(x)))
    dataDropNa["X_split"]=dataDropNa["X_split"].apply(lambda x:"".join(stop_clean(x)))
    # 程序运行慢是因为stopwords太大
    dataDropNa=dataDropNa.dropna(axis=0, how='any')  
    # 最后去空值
    return dataDropNa
 
data = data_pre_handle(data)

In [119]:
#处理测试数据
def process_test(test_name):
    test=pd.read_csv(test_name,sep=",")
    test["X"]=test[["title","content"]].apply(lambda x:"".join([str(x[0]),str(x[1])]),axis=1)
    test["X_split"]=test["X"].apply(lambda x:"".join(character_save(x)))
    test["X_split"]=test["X_split"].apply(lambda x:" ".join(jieba.cut(x)))
    test["X_split"]=test["X_split"].apply(lambda x:"".join(stop_clean(x)))
    
    return test
 
testData=process_test("Test_DataSet.csv")

In [6]:
# import jieba
# #处理训练集，将训练集的文本信息和label信息合并，清洗特殊符号，同时将文本内容进行分词
# def merge_feature_label(feature_name,label_name):
#     feature=pd.read_csv(feature_name,sep=",")
#     label=pd.read_csv(label_name,sep=",")
#     data=feature.merge(label,on='id')
#     data["X"]=data[["title","content"]].apply(lambda x:"".join([str(x[0]),str(x[1])]),axis=1)
#     # 丢掉缺失值
#     dataDropNa=data.dropna(axis=0, how='any')
#     print(dataDropNa.info())
#     dataDropNa["X"]=dataDropNa["X"].apply(lambda x: str(x).replace("\\n","").replace(".","").replace("\n","").replace("　","").replace("↓","").replace("/","").replace("|","").replace(" ",""))
#     dataDropNa["X_split"]=dataDropNa["X"].apply(lambda x:" ".join(jieba.cut(x)))
#     return dataDropNa

In [7]:
# #处理测试数据
# def process_test(test_name):
#     test=pd.read_csv(test_name,sep=",")
#     # apply()方法能劫持另外一个对象的方法，继承另外一个对象的属性 
#     # lambda https://blog.csdn.net/zjuxsl/article/details/79437563
#     #  join() 方法用于将序列中的元素以指定的字符连接生成一个新的字符串。
#     test["X"]=test[["title","content"]].apply(lambda x:"".join([str(x[0]),str(x[1])]),axis=1)
#     # 列名信息
#     print(test.info())
#     # replace(old, new) 方法把字符串中的 old（旧字符串） 替换成 new(新字符串)，如果指定第三个参数max，则替换不超过 max 次。
#     test["X"]=test["X"].apply(lambda x: str(x).replace("\\n","").replace(".","").replace("\n","").replace("　","").replace("↓","").replace("/","").replace("|","").replace(" ",""))
#     # jieba.cut()按照词性切分“我爱中国”切分成我爱中国
#     test["X_split"]=test["X"].apply(lambda x:" ".join(jieba.cut(x)))
#     return test

In [120]:
# dataDropNa=merge_feature_label("Train_DataSet.csv","Train_DataSet_Label.csv")
dataDropNa = data
data_reindex = dataDropNa.reset_index(drop=True)
# data_reindex

In [121]:
data_reindex["X_split"][0:]

0       问责 领导 上 黄镇 党委书记 张涛 宣国 才 真 一手遮天   天 看  人 举报 施  ...
1       江歌 事件 教会 孩子 善良  时 更 懂 保护  去 一年  江歌 悲剧  日 再次 刷 ...
2       绝味 鸭 脖 广告 开 黄腔 引 众怒 双  拼值  双 亿  销售额  中国  全世界 感...
3       央视 曝光 东 一 医药企业  槽罐车 改成 垃圾车 夜间 偷排 高浓度 废水 年  东 高...
4       恶劣 极 央视 都 曝光  南通 东 一 医药企业  槽罐车 改成 洒水车 夜间 偷排 高浓...
5       央视 曝光 南通 一 医药企业 夜间 偷排 高浓度 废水 丢脸 昨晚 央视 一套 晚间新闻 ...
6       粉丝 爆料 五洲 国际 无锡 项目 涉嫌 诈骗 非法 集资  金融街 号   非法 集资 诈...
7       年内 约  锂电 重组 失败 资  高 估值 收购 说 不 摘  中国 电池 联盟  数 显...
8       男子 梦想 一夜 暴富 持 水泥块 砸机 一分钱 都 没取  近日 江苏 扬州 谢  盗取 ...
9       北京 家 法院 供暖 纠纷案件 主体  供暖费 追缴 山海 网 北京   冬天   年 更 ...
10      手机号 开头  注意 看 完   冷汗 都 出  搜 搜狐 原 标题 手机号 开头  注意 ...
11      网红 土坯房 书记 落马  背后 原 标题 网红 土坯房 书记 落马  背后 深耕 卢氏 官...
12      曾  品质 烤肉 两年    难吃  三鼎 甲 曾  尔滨 烤肉 界 位 显赫 两年 前去 ...
13      土俄 军火交易 再 波折 鱼熊 无法 兼顾 土  成美 俄 交锋 牺牲品 搜狐 军事 搜狐网...
14        一  闲鱼  网上交易 平台 购买  一 代付款 服务    国外 网站 上 购物   ...
15      标题 不 签字 不 意 不 反抗  些 人扮 普天庆  样子 好   新兴 商铺 拆迁  态...
16      伙   已 说  教育 废品 文 不 作文 一篇 文章 写  写   不 知道  跑   天...
17      车震 视频 

In [122]:
# testData=process_test("Test_DataSet.csv")
datatest = testData.reset_index(drop=True)

In [146]:
# data_reindex["X_split"][5287]
datatest["X_split"][1407]
 # 2878 3119 3392 3425 5288
    # 1408 3261 6420

'电大    电大   '

In [147]:
data_reindex["X_split"][2877] = data_reindex["X_split"][0]
data_reindex["X_split"][3118] = data_reindex["X_split"][0]
data_reindex["X_split"][3391] = data_reindex["X_split"][0]
data_reindex["X_split"][3424] = data_reindex["X_split"][0]
data_reindex["X_split"][5287] = data_reindex["X_split"][0]

datatest["X_split"][1407] = datatest["X_split"][0]
datatest["X_split"][3260] = datatest["X_split"][0]
datatest["X_split"][6419] = datatest["X_split"][0]


C:\Users\dongy\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\dongy\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\dongy\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users

In [148]:
data_reindex["X_split"][2877]

'问责 领导 上 黄镇 党委书记 张涛 宣国 才 真 一手遮天   天 看  人 举报 施   贴子   举报人 联系 证实  宣 天 中午 请 举报人  枪手 喝酒 后 晚上 才 发  贴子 人 不去 讨 前 二天  举报 相信 总 会  说法  今天 一 看施 全军 年月日 实名 举报 上 黄镇 宣国 才  贴子 仍  锁定 禁止 评 已 正好 一 整年  图片 施 全军 实名 举报 天后 上 黄镇 党委政府 回复 下 图 图片 图片 一年  贴子 再次  网友 顶  后 才 发现 施 天 前 回复 网友  处理结果 竟 下 图 图片 现 责问 张涛 书记 宣国 才  举报   问题 什 时候  答复 宣国 才  举报 后 什  立刻 免  村 书记 职务 什   安排  城 队 吃 空响  却 天天   水泥厂 上班 赚 黑钱   月 水泥 吨 近元 纯利润 还 供不应求 宣国 才 还清 上 黄 政府 担保 借 宣国 才 代付 振 东厂 工资 社保  钱    解 宣国 才 占 人 企业 营  欠税 万元 欠 社保 万元 应 还 欠  职工工资 十万 上 黄 政府 算  宣国 才 担保 还 还  厂 合法 会计  老板  判刑 四 六年 现 服刑 厂子  宣国 才 强占 宣国 才 天 赚 万 净利润 却 外 宣称 天天 亏   老板 刑满 回厂 宣国 才  厂 天天 亏   亏  千万元 甚  亿张 涛 书记  承担 还 上 黄 政府 承担 初   亲  厂 交 宣国 才 生产  希 徐 市长 看 贴 后   批示 批示 违建  民生问题 一样 关注 一下  水泥厂    请 徐 市长 抽 日理万机 空 亲 约 谈 一下 事人 特  位 施 站长 千万 不 听取 一面辞'

In [149]:
def data_to_list(df):
    datas = []
    for content in df["X_split"][0:]:
        dataClean = content.split(" ")
        datas.append(dataClean)
    
    return datas   

In [150]:
X = data_to_list(data_reindex) 

In [151]:
len(X)

7265

In [152]:
Y = data_to_list(datatest) 

In [153]:
len(Y)

7356

In [154]:
import gensim

In [155]:
model = gensim.models.Word2Vec(Y, size=100) # 去掉, iter=10

In [156]:
embeddings_index = dict(zip(model.wv.index2word, model.wv.syn0))

print('Found %s word vectors.' % len(embeddings_index))

C:\Users\dongy\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  if __name__ == '__main__':


Found 44056 word vectors.


In [157]:
def sent2vec(s):
    words = str(s)
    words = word_tokenize(words)
#     words = [w for w in words if not w in stop_words]
    words = [w for w in words]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v ** 2).sum())

In [158]:
x = data_reindex.X_split.values 
y = data_reindex.label.values

In [159]:
xtrain_total = data_reindex.X_split.values 
ytrain_total = data_reindex.label.values
xtest_total = datatest.X_split.values

In [160]:
xtrain_total.shape

(7265,)

In [161]:
xtest_total.shape

(7356,)

In [162]:
xtrain_total_w2v  = [sent2vec(x) for x in tqdm(xtrain_total)]
xtest_total_w2v  = [sent2vec(x) for x in tqdm(xtest_total)]


100%|█████████████████████████████████████████████████████████████████████████████| 7356/7356 [00:09<00:00, 768.58it/s]


In [163]:
xtrain_total_w2v_np = np.array(xtrain_total_w2v)
xtest_total_w2v_np = np.array(xtest_total_w2v)

In [164]:
xtrain_total_w2v_np.shape

(7265, 100)

In [165]:
xtest_total_w2v_np.shape

(7356, 100)

In [166]:
# a = []
# i=0
# for x in xtrain_total_w2v:
    
# #     print(x.tolist())
#     i +=1
#     a += x.tolist()
#     if(len(x.tolist())!=100):
#         print(len(x.tolist()))
#         print(x.tolist())
#         print(i)
    
# #     print(len(x.tolist()))
    

In [167]:

# a = []
# i=0
# for x in xtest_total_w2v:
    
# #     print(x.tolist())
#     i +=1
#     a += x.tolist()
#     if(len(x.tolist())!=100):
#         print(len(x.tolist()))
#         print(x.tolist())
#         print(i)
    
# #     print(len(x.tolist()))
 

In [168]:

def multiclass_logloss(actual, predicted, eps=1e-15):
    """对数损失度量（Logarithmic Loss  Metric）的多分类版本。
    :param actual: 包含actual target classes的数组
    :param predicted: 分类预测结果矩阵, 每个类别都有一个概率
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

In [169]:
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1, silent=False)
clf.fit(xtrain_total_w2v_np, ytrain_total)
predictions = clf.predict_proba(xtest_total_w2v_np)

# print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

In [170]:
preds=predictions
preds=np.argmax(preds,axis=1)
test_pred=pd.DataFrame(preds)
test_pred.columns=["label"]
print(test_pred.shape)
test_pred["id"]=list(testData["id"])
test_pred[["id","label"]].to_csv('1014.csv',index=None)

(7356, 1)


# 深度学习

In [171]:
scl = preprocessing.StandardScaler()
xtrain_total_w2v_np_scl = scl.fit_transform(xtrain_total_w2v_np)
xtest_total_w2v_np_scl = scl.transform(xtest_total_w2v_np)

In [172]:
ytrain_total_enc = np_utils.to_categorical(ytrain_total)
#yvalid_enc = np_utils.to_categorical(yvalid)

In [173]:
model = Sequential()
# 300 改  100
model.add(Dense(300, input_dim=100, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
# 300 改  100
model.add(Dense(300, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())
# 14 改  3
model.add(Dense(3))
model.add(Activation('softmax'))

In [178]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.fit(xtrain_total_w2v_np_scl, y=ytrain_total_enc, batch_size=64, 
          epochs=50, verbose=1)

Epoch 1/50
7265/7265 [==============================] - ETA: 36s - loss: 0.28 - ETA: 2s - loss: 0.2759 - ETA: 0s - loss: 0.280 - ETA: 0s - loss: 0.274 - ETA: 0s - loss: 0.279 - ETA: 0s - loss: 0.286 - 1s 88us/step - loss: 0.2864
Epoch 2/50
7265/7265 [==============================] - ETA: 0s - loss: 0.250 - ETA: 0s - loss: 0.283 - ETA: 0s - loss: 0.282 - ETA: 0s - loss: 0.281 - ETA: 0s - loss: 0.282 - ETA: 0s - loss: 0.287 - 0s 46us/step - loss: 0.2890
Epoch 3/50
7265/7265 [==============================] - ETA: 0s - loss: 0.271 - ETA: 0s - loss: 0.288 - ETA: 0s - loss: 0.275 - ETA: 0s - loss: 0.287 - ETA: 0s - loss: 0.291 - ETA: 0s - loss: 0.291 - 0s 42us/step - loss: 0.2935
Epoch 4/50
7265/7265 [==============================] - ETA: 0s - loss: 0.197 - ETA: 0s - loss: 0.256 - ETA: 0s - loss: 0.247 - ETA: 0s - loss: 0.258 - ETA: 0s - loss: 0.271 - 0s 41us/step - loss: 0.2802
Epoch 5/50
7265/7265 [==============================] - ETA: 0s - loss: 0.243 - ETA: 0s - loss: 0.268 - ETA: 0s

7265/7265 [==============================] - ETA: 0s - loss: 0.103 - ETA: 0s - loss: 0.205 - ETA: 0s - loss: 0.210 - ETA: 0s - loss: 0.219 - ETA: 0s - loss: 0.226 - ETA: 0s - loss: 0.228 - 0s 42us/step - loss: 0.2288
Epoch 37/50
7265/7265 [==============================] - ETA: 0s - loss: 0.147 - ETA: 0s - loss: 0.199 - ETA: 0s - loss: 0.223 - ETA: 0s - loss: 0.226 - ETA: 0s - loss: 0.223 - ETA: 0s - loss: 0.228 - ETA: 0s - loss: 0.230 - 0s 46us/step - loss: 0.2311
Epoch 38/50
7265/7265 [==============================] - ETA: 0s - loss: 0.247 - ETA: 0s - loss: 0.197 - ETA: 0s - loss: 0.213 - ETA: 0s - loss: 0.221 - ETA: 0s - loss: 0.230 - 0s 36us/step - loss: 0.2318
Epoch 39/50
7265/7265 [==============================] - ETA: 0s - loss: 0.252 - ETA: 0s - loss: 0.219 - ETA: 0s - loss: 0.228 - ETA: 0s - loss: 0.233 - ETA: 0s - loss: 0.230 - 0s 39us/step - loss: 0.2330
Epoch 40/50
7265/7265 [==============================] - ETA: 0s - loss: 0.201 - ETA: 0s - loss: 0.214 - ETA: 0s - loss:

In [179]:
predictions = model.predict_proba(xtest_total_w2v_np_scl)
preds=predictions
preds=np.argmax(preds,axis=1)
test_pred=pd.DataFrame(preds)
test_pred.columns=["label"]
print(test_pred.shape)
test_pred["id"]=list(testData["id"])
test_pred[["id","label"]].to_csv('0000000050ci.csv',index=None)
# 过拟合。

(7356, 1)


In [67]:
earlystop = EarlyStopping(monitor='loss', min_delta=0, patience=5, verbose=0, mode='auto')
model.fit(xtrain_pad, y=ytrain_total_enc, batch_size=512, epochs=200, 
          verbose=1, callbacks=[earlystop])

ValueError: Error when checking input: expected dense_16_input to have shape (100,) but got array with shape (70,)

In [45]:
token = text.Tokenizer(num_words=None)
max_len = 70

token.fit_on_texts(list(xtrain_total) + list(xtest_total))
xtrain_seq = token.texts_to_sequences(xtrain_total)
xtest_seq = token.texts_to_sequences(xtest_total)

In [46]:
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xtest_pad = sequence.pad_sequences(xtest_seq, maxlen=max_len)

word_index = token.word_index

In [47]:
embedding_matrix = np.zeros((len(word_index) + 1, 100))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|█████████████████████████████████████████████████████████████████████| 275101/275101 [00:00<00:00, 1145793.13it/s]


In [48]:
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     100,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.fit(xtrain_pad, y=ytrain_total_enc, batch_size=512, epochs=100, verbose=1)

Epoch 1/100
7265/7265 [==============================] - ETA: 8s - loss: 1.205 - ETA: 6s - loss: 1.119 - ETA: 4s - loss: 1.073 - ETA: 4s - loss: 1.053 - ETA: 3s - loss: 1.034 - ETA: 3s - loss: 1.044 - ETA: 2s - loss: 1.046 - ETA: 2s - loss: 1.047 - ETA: 1s - loss: 1.043 - ETA: 1s - loss: 1.037 - ETA: 1s - loss: 1.031 - ETA: 0s - loss: 1.024 - ETA: 0s - loss: 1.018 - ETA: 0s - loss: 1.013 - 5s 683us/step - loss: 1.0107
Epoch 2/100
7265/7265 [==============================] - ETA: 4s - loss: 0.950 - ETA: 4s - loss: 0.962 - ETA: 3s - loss: 0.955 - ETA: 3s - loss: 0.959 - ETA: 3s - loss: 0.952 - ETA: 2s - loss: 0.947 - ETA: 2s - loss: 0.949 - ETA: 2s - loss: 0.951 - ETA: 1s - loss: 0.952 - ETA: 1s - loss: 0.954 - ETA: 1s - loss: 0.952 - ETA: 0s - loss: 0.950 - ETA: 0s - loss: 0.949 - ETA: 0s - loss: 0.949 - 5s 660us/step - loss: 0.9491
Epoch 3/100
7265/7265 [==============================] - ETA: 4s - loss: 0.940 - ETA: 4s - loss: 0.945 - ETA: 3s - loss: 0.930 - ETA: 3s - loss: 0.938 - ETA

7265/7265 [==============================] - ETA: 4s - loss: 0.772 - ETA: 4s - loss: 0.745 - ETA: 4s - loss: 0.732 - ETA: 4s - loss: 0.735 - ETA: 3s - loss: 0.741 - ETA: 3s - loss: 0.733 - ETA: 2s - loss: 0.732 - ETA: 2s - loss: 0.726 - ETA: 2s - loss: 0.723 - ETA: 1s - loss: 0.724 - ETA: 1s - loss: 0.719 - ETA: 0s - loss: 0.724 - ETA: 0s - loss: 0.720 - ETA: 0s - loss: 0.723 - 6s 789us/step - loss: 0.7223
Epoch 39/100
7265/7265 [==============================] - ETA: 5s - loss: 0.727 - ETA: 5s - loss: 0.731 - ETA: 4s - loss: 0.737 - ETA: 3s - loss: 0.728 - ETA: 3s - loss: 0.733 - ETA: 3s - loss: 0.725 - ETA: 2s - loss: 0.723 - ETA: 2s - loss: 0.723 - ETA: 2s - loss: 0.723 - ETA: 1s - loss: 0.720 - ETA: 1s - loss: 0.720 - ETA: 0s - loss: 0.718 - ETA: 0s - loss: 0.719 - ETA: 0s - loss: 0.721 - 6s 796us/step - loss: 0.7199
Epoch 40/100
7265/7265 [==============================] - ETA: 5s - loss: 0.724 - ETA: 4s - loss: 0.735 - ETA: 4s - loss: 0.711 - ETA: 4s - loss: 0.716 - ETA: 3s - los

7265/7265 [==============================] - ETA: 5s - loss: 0.618 - ETA: 5s - loss: 0.642 - ETA: 4s - loss: 0.652 - ETA: 4s - loss: 0.655 - ETA: 3s - loss: 0.655 - ETA: 3s - loss: 0.658 - ETA: 3s - loss: 0.655 - ETA: 2s - loss: 0.664 - ETA: 2s - loss: 0.657 - ETA: 1s - loss: 0.658 - ETA: 1s - loss: 0.657 - ETA: 0s - loss: 0.652 - ETA: 0s - loss: 0.653 - ETA: 0s - loss: 0.654 - 6s 832us/step - loss: 0.6543
Epoch 77/100
7265/7265 [==============================] - ETA: 5s - loss: 0.655 - ETA: 5s - loss: 0.652 - ETA: 4s - loss: 0.651 - ETA: 4s - loss: 0.662 - ETA: 3s - loss: 0.666 - ETA: 3s - loss: 0.661 - ETA: 2s - loss: 0.654 - ETA: 2s - loss: 0.650 - ETA: 2s - loss: 0.648 - ETA: 1s - loss: 0.650 - ETA: 1s - loss: 0.651 - ETA: 0s - loss: 0.649 - ETA: 0s - loss: 0.650 - ETA: 0s - loss: 0.649 - 6s 818us/step - loss: 0.6488
Epoch 78/100
7265/7265 [==============================] - ETA: 5s - loss: 0.619 - ETA: 4s - loss: 0.614 - ETA: 4s - loss: 0.615 - ETA: 4s - loss: 0.624 - ETA: 3s - los

In [49]:
predictions = model.predict_proba(xtest_pad)
preds=predictions
preds=np.argmax(preds,axis=1)
test_pred=pd.DataFrame(preds)
test_pred.columns=["label"]
print(test_pred.shape)
test_pred["id"]=list(testData["id"])
test_pred[["id","label"]].to_csv('0000000002.csv',index=None)

(7356, 1)


In [51]:
# 基于前面训练的Word2vec词向量，构建1个2层的GRU模型
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     100,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(GRU(100, dropout=0.3, recurrent_dropout=0.3, return_sequences=True))
model.add(GRU(100, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [52]:
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtrain_pad, y=ytrain_total_enc, batch_size=512, epochs=100, 
          verbose=1, callbacks=[earlystop])

Epoch 1/100
7265/7265 [==============================] - ETA: 19s - loss: 43655880.000 - ETA: 15s - loss: 45860890.000 - ETA: 14s - loss: 52743210.666 - ETA: 13s - loss: 52235624.000 - ETA: 11s - loss: 50852563.200 - ETA: 10s - loss: 47673459.000 - ETA: 9s - loss: 45607707.142 - ETA: 8s - loss: 42679960.00 - ETA: 6s - loss: 40113673.11 - ETA: 5s - loss: 37131726.10 - ETA: 4s - loss: 34855042.36 - ETA: 2s - loss: 32665632.50 - ETA: 1s - loss: 30706596.30 - ETA: 0s - loss: 28810627.21 - 20s 3ms/step - loss: 28485104.1904


C:\Users\dongy\Anaconda3\envs\tensorflow\lib\site-packages\keras\callbacks\callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


Epoch 2/100
7265/7265 [==============================] - ETA: 18s - loss: 3698575.75 - ETA: 18s - loss: 3792359.87 - ETA: 16s - loss: 4049522.58 - ETA: 15s - loss: 4079587.68 - ETA: 13s - loss: 3705752.55 - ETA: 12s - loss: 3401453.06 - ETA: 10s - loss: 3581701.48 - ETA: 9s - loss: 3295170.9375 - ETA: 7s - loss: 3073776.736 - ETA: 6s - loss: 2903420.362 - ETA: 4s - loss: 2746676.431 - ETA: 3s - loss: 2757638.145 - ETA: 1s - loss: 2673727.211 - ETA: 0s - loss: 2679010.571 - 22s 3ms/step - loss: 2696532.2586
Epoch 3/100
7265/7265 [==============================] - ETA: 20s - loss: 432360.187 - ETA: 19s - loss: 665498.406 - ETA: 17s - loss: 679408.166 - ETA: 15s - loss: 844927.750 - ETA: 14s - loss: 802354.750 - ETA: 12s - loss: 849816.604 - ETA: 11s - loss: 896366.428 - ETA: 9s - loss: 845618.867 - ETA: 8s - loss: 814542.53 - ETA: 6s - loss: 778539.36 - ETA: 4s - loss: 756051.59 - ETA: 3s - loss: 770522.64 - ETA: 1s - loss: 815553.74 - ETA: 0s - loss: 800395.92 - 22s 3ms/step - loss: 848

KeyboardInterrupt: 

In [ ]:
predictions = model.predict_proba(xtest_pad)
preds=predictions
preds=np.argmax(preds,axis=1)
test_pred=pd.DataFrame(preds)
test_pred.columns=["label"]
print(test_pred.shape)
test_pred["id"]=list(testData["id"])
test_pred[["id","label"]].to_csv('0000000003.csv',index=None)

In [53]:
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     100,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(Bidirectional(LSTM(100, dropout=0.3, recurrent_dropout=0.3)))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [55]:
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtrain_pad, y=ytrain_total_enc, batch_size=512, epochs=100, 
          verbose=1, callbacks=[earlystop])

Epoch 1/100
7265/7265 [==============================] - ETA: 30s - loss: 1.08 - ETA: 24s - loss: 1.01 - ETA: 21s - loss: 1.01 - ETA: 18s - loss: 0.98 - ETA: 16s - loss: 0.98 - ETA: 14s - loss: 0.97 - ETA: 13s - loss: 0.96 - ETA: 11s - loss: 0.96 - ETA: 9s - loss: 0.9657 - ETA: 7s - loss: 0.958 - ETA: 5s - loss: 0.955 - ETA: 3s - loss: 0.955 - ETA: 2s - loss: 0.951 - ETA: 0s - loss: 0.950 - 26s 4ms/step - loss: 0.9491


C:\Users\dongy\Anaconda3\envs\tensorflow\lib\site-packages\keras\callbacks\callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


Epoch 2/100
7265/7265 [==============================] - ETA: 24s - loss: 0.90 - ETA: 22s - loss: 0.89 - ETA: 20s - loss: 0.90 - ETA: 19s - loss: 0.89 - ETA: 17s - loss: 0.88 - ETA: 15s - loss: 0.87 - ETA: 13s - loss: 0.87 - ETA: 11s - loss: 0.88 - ETA: 9s - loss: 0.8840 - ETA: 8s - loss: 0.881 - ETA: 6s - loss: 0.880 - ETA: 4s - loss: 0.877 - ETA: 2s - loss: 0.872 - ETA: 0s - loss: 0.870 - 28s 4ms/step - loss: 0.8700
Epoch 3/100
6656/7265 [==========================>...] - ETA: 26s - loss: 0.77 - ETA: 24s - loss: 0.78 - ETA: 22s - loss: 0.79 - ETA: 20s - loss: 0.82 - ETA: 18s - loss: 0.82 - ETA: 16s - loss: 0.82 - ETA: 14s - loss: 0.81 - ETA: 12s - loss: 0.81 - ETA: 10s - loss: 0.81 - ETA: 8s - loss: 0.8167 - ETA: 6s - loss: 0.816 - ETA: 4s - loss: 0.817 - ETA: 2s - loss: 0.8196

KeyboardInterrupt: 

In [ ]:
predictions = model.predict_proba(xtest_pad)
preds=predictions
preds=np.argmax(preds,axis=1)
test_pred=pd.DataFrame(preds)
test_pred.columns=["label"]
print(test_pred.shape)
test_pred["id"]=list(testData["id"])
test_pred[["id","label"]].to_csv('0000000004.csv',index=None)